## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-03-18-47-02 +0000,bbc,Victims of mis-sold car finance could get less...,https://www.bbc.com/news/articles/cgjy29zql25o...
1,2025-08-03-18-44-45 +0000,nyt,A Former American Soldier Finds a Home in Mexi...,https://www.nytimes.com/2025/08/03/world/ameri...
2,2025-08-03-18-42-31 +0000,nypost,Whale killed off Jersey Shore after bashing bo...,https://nypost.com/2025/08/03/us-news/whale-ki...
3,2025-08-03-18-39-45 +0000,nypost,Montana bar shooting victims named — as top co...,https://nypost.com/2025/08/03/us-news/montana-...
4,2025-08-03-18-38-24 +0000,nyt,Trump’s Tariffs Are Making Money. That May Mak...,https://www.nytimes.com/live/2025/08/03/us/tru...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2303/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
35,trump,31
31,new,8
74,he,8
162,russian,8
25,bar,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
155,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...,83
75,2025-08-03-14-22-11 +0000,nypost,Trump rips ‘sleazebag’ Charlamagne tha God aft...,https://nypost.com/2025/08/03/us-news/trump-ri...,81
143,2025-08-03-00-48-00 +0000,wsj,Erika McEntarfer led the Bureau of Labor Stati...,https://www.wsj.com/politics/policy/the-econom...,63
55,2025-08-03-15-21-03 +0000,nyt,India Will Buy Russian Oil Despite Trump’s Thr...,https://www.nytimes.com/2025/08/02/world/asia/...,56
118,2025-08-03-08-49-11 +0000,nypost,India to maintain Russian oil import despite T...,https://nypost.com/2025/08/03/world-news/india...,55


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
155,83,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...
93,48,2025-08-03-10-34-36 +0000,nypost,Ukrainian drone attack sparks massive fire at ...,https://nypost.com/2025/08/03/world-news/ukrai...
3,43,2025-08-03-18-39-45 +0000,nypost,Montana bar shooting victims named — as top co...,https://nypost.com/2025/08/03/us-news/montana-...
123,30,2025-08-03-07-39-06 +0000,nypost,US envoy tells Israeli hostage families he’s w...,https://nypost.com/2025/08/03/world-news/us-en...
143,29,2025-08-03-00-48-00 +0000,wsj,Erika McEntarfer led the Bureau of Labor Stati...,https://www.wsj.com/politics/policy/the-econom...
83,27,2025-08-03-12-51-41 +0000,nypost,Woman killed near California park after being ...,https://nypost.com/2025/08/03/us-news/pack-of-...
131,25,2025-08-03-04-27-43 +0000,bbc,"Their children can't eat, speak or walk - so f...",https://www.bbc.com/news/articles/c39d0zj0110o...
137,24,2025-08-03-02-00-00 +0000,wsj,The build-out of AI infrastructure is costing ...,https://www.wsj.com/economy/the-ai-booms-hidde...
31,23,2025-08-03-16-55-11 +0000,nypost,JPMorgan employees gripe about paying for ‘cut...,https://nypost.com/2025/08/03/business/jpm-emp...
122,23,2025-08-03-07-55-15 +0000,nypost,Aussie running for Boston suburb mayor says th...,https://nypost.com/2025/08/03/us-news/massachu...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
